<hr style="border:2px solid #0176DE"> </hr>
<center><h1 style="color:#173F8A;"> EMI 2024 - CMM Chile</h1></center>
<hr style="border:2px solid #0176DE"> </hr>
<h3 style="color:#173F8A;text-align:right;"> Profesores: &nbsp;Nicolás Barnafi<br>Manuel A. Sánchez<br></h3>

<h3 style="color:#03122E;text-align:right;"> 
    Centro de Modelamiento Matemático <br> 
    Instituto de Ingeniería Matemática y Computacional - IMC UC<br>  
</h3>

<hr style="border:2px solid #03122E"> </hr>
<center><h1 style="color:#173F8A;"> Módulo 5: The wave equations - NGSolve</h1></center> 
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC:
Primaria:
celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 
Secundaria
gris oscuro: #707070
-->

In [33]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
# from netgen.occ import *
from netgen.geom2d import CSG2d, Circle, Rectangle

from netgen.webgui import Draw as DrawGeo
from ngsolve.krylovspace import GMResSolver
from time import perf_counter as time


In [40]:

tend = 5
maxh = 0.1


def test1mesh(maxh=0.2):
    geo = CSG2d()
    # define some primitives
    circle = Circle( center=(0,0), radius=0.25, mat="circle", bc="bc_circle" )
    rect = Rectangle( pmin=(-1,-1), pmax=(1,1), mat="out", bc="bc_rect" )
    # use operators +, - and * for union, difference and intersection operations
    domain1 = circle * rect
    domain2 = (rect-circle).Mat('out')
    # add top level objects to geometry
    geo.Add(domain1)
    geo.Add(domain2)
    # generate mesh
    m = geo.GenerateMesh(maxh=maxh)
    mesh = Mesh(m)
    mesh.Curve(3)
    return mesh
    
def solveWaveEq(maxh, method, draw=False): #for maxh in [0.2, 0.1, 0.05, 0.025]: 
#     shape = Rectangle(2,2).Face().Move((-1,-1,0))
    
    ic = 2
    if ic == 1:
#         mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=maxh))
        geo = CSG2d()
        rect = Rectangle( pmin=(-1,-1), pmax=(1,1), mat="out", bc="bc_rect" )
        geo.Add(rect)
        m = geo.GenerateMesh(maxh=maxh)
        mesh = Mesh(m)
        rr = x**2 + y**2 
#         def tanh(xx): return (exp(xx) - exp(-xx)) / (exp(xx) + exp(-xx))
#         u0 = 1 - tanh(10 * rr) #(exp(rr) - exp(-rr)) / (exp(rr) + exp(-rr))
        u0 = IfPos( x*x+y*y-(0.25)**2, 0, 1.0)
    elif ic == 2:
        
        mesh = test1mesh(maxh)
        u0 = mesh.RegionCF(VOL, dict(circle=1, out=0))
#         Draw(u0,mesh)
    
    fes = H1(mesh, order=1)
    X = fes * fes
    (u,v), (ut, vt) = X.TnT()
    t = 0.0
    dt = 0.01
    idt = 1/dt
    mu = 0.1
    
    a = BilinearForm(X, symmetric=False)
    a += idt * v * vt * dx + idt * u * ut * dx - v * ut * dx #mu*grad(u)*grad(vt)*dx 
    aP = Preconditioner(a, "local")
    a.Assemble()
    
    g = BilinearForm(X, symmetric=False)
    g += -idt * v * vt * dx - idt * u * ut * dx + mu*grad(u)*grad(vt)*dx 
    g.Assemble()
    
    gf = GridFunction(X)
    gfu, gfv = gf.components
    
    C = 10.
    
    gfu.Set(u0)
    if draw:
        scene = Draw(u0, mesh)
    
    def TimeStepping(inv, t0 = 0, tend = 0.2,
                     saveEvery=10):
        cnt = 0; t = t0
        gfut = GridFunction(gfu.space,multidim=0)
        gfut.AddMultiDimComponent(gfu.vec)
        while t < tend - 0.5 * dt:
            res = -g.mat * gf.vec
            gf.vec.data = inv * res
            print("\r{:1.4f}".format(t),end="")
            if draw: scene.Redraw()
            if cnt % saveEvery == 0:
                gfut.AddMultiDimComponent(gfu.vec)
            cnt += 1; t = cnt * dt
        return gfut
    
    t0 = time()
    if method == "direct": 
        inv = a.mat.Inverse()
    elif method == "iterative":
        inv = GMResSolver(a.mat, aP.mat, printrates=None, maxiter=200, tol=1e-6)
    gfu.Set(u0)
    gfv.Set(0.0)
    gfut = TimeStepping(inv, tend=tend)
    tf = time() - t0
    if draw:
        Draw(gfut, mesh, interpolate_multidim=True, animate=True)
    return tf


t0 = time()
# solveWaveEq(maxh, 'direct', draw=False)
t_direct = time() - t0

t0 = time()
solveWaveEq(maxh, 'iterative', draw=True)
t_iter = time() - t0

print("\nSolve time direct={:2.3f}".format(t_direct))
print("Solve time iterative={:2.3f}".format(t_iter))
print("Ratio:", t_iter/t_direct)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

4.9900

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…


Solve time direct=0.000
Solve time iterative=12.067
Ratio: 162186.1540755925


In [36]:
from netgen.geom2d import CSG2d, Circle, Rectangle

geo = CSG2d()

# define some primitives
circle = Circle( center=(0,0), radius=0.25, mat="circle", bc="bc_circle" )
rect = Rectangle( pmin=(-1,-1), pmax=(1,1), mat="out", bc="bc_rect" )

# use operators +, - and * for union, difference and intersection operations
domain1 = circle * rect
# domain2.Mat("mat3").Maxh(0.1) # change domain name and maxh
domain2 = (rect-circle).Mat('out')

# add top level objects to geometry
geo.Add(domain1)
geo.Add(domain2)

# generate mesh
m = geo.GenerateMesh(maxh=0.3)

# use NGSolve just for visualization
from ngsolve.webgui import Draw
from ngsolve import Mesh, VOL
mesh = Mesh(m)
mesh.Curve(3)
u0 = mesh.RegionCF(VOL, dict(circle=1, out=0))
Draw(u0, mesh)
type(cf)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

NameError: name 'cf' is not defined

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene